In [1]:
%cd C:\Users\user\Drive\s-hero\code\yolov5_classifier
!pip install -qr requirements.txt

C:\Users\user\Drive\s-hero\code\yolov5_classifier


In [11]:
import os
import sys
from pathlib import Path
from glob import glob
import gc
import yaml

import matplotlib 
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split

import torch 
from torch.utils.data import Dataset, DataLoader, RandomSampler
from torch.utils.data.dataset import random_split

from PIL import Image
import PIL
from tqdm import tqdm
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms as T
from pytorch_lightning import LightningModule, LightningDataModule, Trainer, seed_everything
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torchmetrics.functional import accuracy
from pytorch_model_summary import summary as pms
from torchinfo import summary as info
from torchsummary import summary 
from yolov5_classifier import *
from yolov5_classifier.train import *
from yolov5_classifier.val import *
from yolov5_classifier.detect import *
from yolov5_classifier.export import *
from yolov5_classifier.hubconf import *

from yolov5_classifier.models import *
from yolov5_classifier.models.common import *
from yolov5_classifier.models.experimental import *
from yolov5_classifier.models.yolo import Model
from yolov5_classifier.models.yolo import *
from yolov5_classifier.utils import *

import random 
random.seed(7)

USE_CUDA = torch.cuda.is_available()
device = torch.device('cuda:0' if USE_CUDA else 'cpu')
print('pytorch version : ',torch.__version__)
print('device:',device)
print('gpus:', torch.cuda.device_count())
print('graphic name:', torch.cuda.get_device_name())

pytorch version :  1.9.1
device: cuda:0
gpus: 1
graphic name: NVIDIA GeForce RTX 3080 Ti


In [36]:
with open(r'C:\Users\user\Drive\s-hero\Code\yolov5_classifier\models\yolov5m.yaml', 'r') as f:
    v5m = yaml.load(f, Loader=yaml.FullLoader)

with open(r'C:\Users\user\Drive\s-hero\Code\yolov5\models\yolov5m.yaml', 'r') as f:
    v5m_org = yaml.load(f, Loader=yaml.FullLoader)

org_model = Model(v5m_org, 3, 8)
model = Model(v5m, 3, 8)

In [53]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5m', pretrained=True, autoshape=False)
model.model = model.model[:8]
m = model.model[-1]  # last layer
ch = m.conv.in_channels if hasattr(m, 'conv') else sum([x.in_channels for x in m.m])  # ch into module
c = Classify(ch, 8)  # Classify()
c.i, c.f, c.type = m.i, m.f, 'models.common.Classify'  # index, from, type
model.model[-1] = c  # replace
for p in model.parameters():
        p.requires_grad = True

Using cache found in C:\Users\user/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  0c93333 torch 1.9.1 CUDA:0 (NVIDIA GeForce RTX 3080 Ti, 12288.0MB)

Fusing layers... 
Model Summary: 290 layers, 21172173 parameters, 0 gradients, 49.0 GFLOPs


In [55]:
#info(model, (32, 3, 1020, 1980), device='cpu')

Layer (type:depth-idx)                        Output Shape              Param #
Model                                         --                        --
├─Sequential: 1                               --                        --
│    └─Conv: 2-1                              [32, 48, 510, 990]        --
│    │    └─Conv2d: 3-1                       [32, 48, 510, 990]        5,232
│    │    └─SiLU: 3-2                         [32, 48, 510, 990]        --
│    └─Conv: 2-2                              [32, 96, 255, 495]        --
│    │    └─Conv2d: 3-3                       [32, 96, 255, 495]        41,568
│    │    └─SiLU: 3-4                         [32, 96, 255, 495]        --
│    └─C3: 2-3                                [32, 96, 255, 495]        --
│    │    └─Conv: 3-5                         [32, 48, 255, 495]        4,656
│    │    └─Sequential: 3-6                   [32, 48, 255, 495]        46,272
│    │    └─Conv: 3-7                         [32, 48, 255, 495]        4,656
│  

In [56]:
#}ckpt = torch.load(r'C:\Users\user\Drive\s-hero\Code\yolov5\runs\train\m_b32_300epoch_evo\weights\best.pt')
ckpt = torch.load(r'C:\Users\user\Drive\s-hero\Code\yolov5_classifier\runs\train\m_b16_300epoch\weights\best.pt')
weight = ckpt['model'].state_dict()

In [42]:
weight.keys()

odict_keys(['model.0.conv.weight', 'model.0.conv.bias', 'model.1.conv.weight', 'model.1.conv.bias', 'model.2.cv1.conv.weight', 'model.2.cv1.conv.bias', 'model.2.cv2.conv.weight', 'model.2.cv2.conv.bias', 'model.2.cv3.conv.weight', 'model.2.cv3.conv.bias', 'model.2.m.0.cv1.conv.weight', 'model.2.m.0.cv1.conv.bias', 'model.2.m.0.cv2.conv.weight', 'model.2.m.0.cv2.conv.bias', 'model.2.m.1.cv1.conv.weight', 'model.2.m.1.cv1.conv.bias', 'model.2.m.1.cv2.conv.weight', 'model.2.m.1.cv2.conv.bias', 'model.3.conv.weight', 'model.3.conv.bias', 'model.4.cv1.conv.weight', 'model.4.cv1.conv.bias', 'model.4.cv2.conv.weight', 'model.4.cv2.conv.bias', 'model.4.cv3.conv.weight', 'model.4.cv3.conv.bias', 'model.4.m.0.cv1.conv.weight', 'model.4.m.0.cv1.conv.bias', 'model.4.m.0.cv2.conv.weight', 'model.4.m.0.cv2.conv.bias', 'model.4.m.1.cv1.conv.weight', 'model.4.m.1.cv1.conv.bias', 'model.4.m.1.cv2.conv.weight', 'model.4.m.1.cv2.conv.bias', 'model.4.m.2.cv1.conv.weight', 'model.4.m.2.cv1.conv.bias', 'mod

In [57]:
model.load_state_dict(weight)

<All keys matched successfully>